プログラムの構成
#1.事前設定:oandapyのインストール, googlecolabのドライブマウント, setup_fxconn()
#2.為替相場から値を取得:fetch_fxvalue()
#3.データ成形し、特徴量抽出:calc_feat()
#4.モデル学習:train_model()
#5.将来の為替変動予測:predict()
#6.予測結果をもとに取引の戦略を立てる:build_strategy()
#7.取引:transaction_with_strategy()

In [4]:
#1.事前設定
#oandapyインストール
pip install git+https://github.com/oanda/oandapy.git

SyntaxError: invalid syntax (<ipython-input-4-0799e0ff2841>, line 3)

In [ ]:
#google collaboration上で、Google ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [17]:
#2. 為替相場から値を取得:fetch_fxvalue()
# API接続設定のファイルを読み込む
import configparser
import oandapy
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pytz
import datetime
from datetime import datetime, timedelta


#TODO: 外部から読み込む形にしたい
#path_oanda='./drive/My Drive/oandapy' #google colac用
path_oanda='..'
path_config=path_oanda+'/data/config.txt'

config = configparser.ConfigParser()
config.read(path_config)

#TODO: グローバル変数にしたくない
instrument = config['oanda']['instrument']
granularity = config['oanda']['granularity']
count = int(config['oanda']['count'])
api_key = config['oanda']['api_key']
account_id = config['oanda']['account_id']


def create_api_fxconn(path_config):
# API接続
    oanda = oandapy.API(environment="practice", access_token=api_key)
    return oanda


def iso_jp(iso):
#日付の調整する
#文字列 -> datetime
     date = None
     try:
         date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
         date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
     except ValueError:
         try:
             date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
             date = dt.astimezone(pytz.timezone("Asia/Tokyo"))
         except ValueError:
             pass
     return date

  
def create_df_fxvalue_latesthistory_rawdata(oanda):
#生データのdfを生成
    instrument, granularity, count = (config['oanda']['instrument'], 
                                      config['oanda']['granularity'], 
                                      int(config['oanda']['count']))
    json_response = oanda.get_history(instrument=instrument, 
                                      granularity=granularity, 
                                      count=count)
    df_response = pd.DataFrame(json_response["candles"])
    df_response["time"] = df_response["time"].apply(lambda x: iso_jp(x))
    return df_response

#-----------------main-----------------------------

def fetch_fxvalue(path_config):
#2のメイン関数, 為替の値を取得
    instrument, granularity, count = (config['oanda']['instrument'], 
                                      config['oanda']['granularity'], 
                                      int(config['oanda']['count']))
    oanda = create_api_fxconn(path_config=path_config)
    df_fxvalue_latesthistory_rawdata = create_df_fxvalue_latesthistory_rawdata(oanda)
    return df_fxvalue_latesthistory_rawdata 
    

In [20]:
#2 テスト
path_oanda='..'
path_config=path_oanda+'/data/config.txt'
tmp = fetch_fxvalue(path_config)
tmp.head()

,closeAsk,closeBid,complete,highAsk,highBid,lowAsk,lowBid,openAsk,openBid,time,volume
0,109.984,109.971,True,109.993,109.983,109.983,109.971,109.983,109.971,2020-01-21 18:15:00+09:00,34
1,109.985,109.974,True,109.989,109.977,109.984,109.972,109.985,109.973,2020-01-21 18:16:00+09:00,41
2,109.981,109.968,True,109.984,109.972,109.972,109.960,109.984,109.972,2020-01-21 18:17:00+09:00,51
3,109.987,109.975,True,109.991,109.979,109.978,109.966,109.982,109.970,2020-01-21 18:18:00+09:00,47
4,109.979,109.967,True,109.990,109.977,109.977,109.966,109.985,109.974,2020-01-21 18:19:00+09:00,31


In [49]:
#3. 特徴量抽出:feat_calc()
# API接続設定のファイルを読み込む

def data_maker(data, window_len):
    data_lstm_in=[]
    if len(data)==window_len:
        temp = data[:window_len].copy()
        temp = temp / temp.iloc[0] - 1
        data_lstm_in.append(temp)
    for i in range(len(data) - window_len):
        temp = data[i:(i + window_len)].copy()
        temp = temp / temp.iloc[0] - 1
        data_lstm_in.append(temp)
    return data_lstm_in


def pd_to_np(data_lstm_in):
#うんこみたいな関数すぎる、変えたい
#もともとdf？listになってるんやけど
    data_lstm_in = [np.array(data_lstm_input) for data_lstm_input in data_lstm_in]  #array のリスト
    data_lstm_in = np.array(data_lstm_in) #np.array
    return data_lstm_in


def extract_feature(df_rawdata, ratio_num_traindata):
#特徴量抽出->ここをいじります
#ratio_num_traindataは訓練データと検証用データの分割比
#TODO: ratio_num_traindataは引数で与えたい

    #1. 特徴量算出に必要なカラムを設定
    df = df_rawdata[['time', 'openAsk']]
    df.columns = ['time', 'open']

    #2. 予測に使うデータ数を設定
    window_len = 10
    #3. 訓練データと検証用データを所定の割合で分割
    #TODO: ごちゃごちゃしているので、書き換えたい
    train, test = df[df.index < int(len(df)*ratio_num_traindata)], \
                    df[df.index >= int(len(df)*ratio_num_traindata)]
    latest = test[:window_len]
    del train['time']
    del test['time']
    del latest['time']
    length = len(test)- window_len
    
    #モデル入力、モデル出力用にデータ成形
    ##ここを直して、特徴量抽出のアルゴリズム考える
    #TODO: data_makerとかpd_to_npとかいう意味わからん関数書き換えたい
    #TODO: latestがよくわからん
    train_lstm_in = data_maker(train, window_len)
    lstm_train_out = (train['open'][window_len:].values / train['open'][:-window_len].values)-1
    test_lstm_in = data_maker(test, window_len)
    lstm_test_out = (test['open'][window_len:].values / test['open'][:-window_len].values)-1
    latest_lstm_in = data_maker(latest, window_len)
    train_lstm_in = pd_to_np(train_lstm_in)
    test_lstm_in = pd_to_np(test_lstm_in)
    latest_lstm_in = pd_to_np(latest_lstm_in)
    
    #入出力の名前とかが意味わからなかったので、dictで管理してる
    X_dict = {'train':train_lstm_in, 'test':test_lstm_in, 'latest':latest_lstm_in}
    y_dict = {'train':lstm_train_out, 'test':lstm_test_out}

    return X_dict, y_dict

In [50]:
#3テスト
X_dict, y_dict = extract_feature(tmp, ratio_num_traindata=0.8)

In [52]:
#4 モデル学習: train_model()
#TODO なんもやってない


def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
# モデル設定->ここを工夫
    model = Sequential()
 
    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
 
    model.compile(loss=loss, optimizer=optimizer)
    return model

#TODO: ランダムシードの設定->文頭に入れるべき
np.random.seed(202)
 
# 初期モデルの構築
X_train, y_train = X_dict['train'], y_dict['train']
#TODO LSTMのセルの数をneuronsという名前から変更すべき
#TODO ネットワークの設定を、別途configファイルかargs.parserで引数として外部設定できるようにする
model = build_model(X_train, output_size=1, neurons = 20)
history = model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2, shuffle=True)

# #未来の値
#TODO: 何も変えていない
# empty = []
# future_array = np.array(empty)
# for i in range(length):
#     pred = (((np.transpose(yen_model.predict(latest_lstm_in))+1) * latest['open'].values[0])[0])[0]
#     future_array= np.append(future_array,pred)
#     data ={'open':[pred]}
#     df1 = pd.DataFrame(data)
#     latest =pd.concat([latest,df1],axis=0)
#     latest.index = range(0,window_len+1)
#     latest = latest.drop(0,axis=0)
#     latest_lstm_in =pd_to_np(latest_lstm_in)
    


W0126 14:28:15.028591 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0126 14:28:16.147869 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0126 14:28:16.441787 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0126 14:28:17.533325 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_defa

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
#4テスト


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import oandapy
import configparser
import pytz
import datetime
from datetime import datetime, timedelta

from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

#main
def main():
    